# PRL and eye-tracking 

In [1]:
import os
import pandas as pd
import numpy as np
from functools import reduce
from pandas.api.types import CategoricalDtype
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import sys
import datetime


In [2]:
now = datetime.datetime.now()
formatted_date = now.strftime("%Y-%m-%d %H:%M:%S")

print("Written by Corrado Caudek")
print("Last change:", formatted_date)

Written by Corrado Caudek
Last change: 2023-03-18 11:07:20


In [20]:
%run helpers_cc_prlet.ipynb

In [13]:
%pwd

'/Users/corrado/_repositories/prl_eye_movements/scripts'

Save the folder names contained in the directory `thesis_marta`.

In [21]:
# Set the directory path
path_to_project_folder = os.path.join(
        "..",
        "data",
        "raw",
        "thesis_marta"
    )
print(path_to_project_folder)

../data/raw/thesis_marta


Get the names of the subjects folders.

In [22]:
names_of_subjects_folders = get_names_of_subj_folders(path_to_project_folder)
print(names_of_subjects_folders)

['fr_va_1960_08_18_538_u', 'ma_va_1998_07_04_538_d']


Select one subject.

In [23]:
# SUBJECT_INDEX identifies the elements of the `folder` list.
SUBJECT_INDEX = 1

For the selected subject, get the name of the file in the `biographical_data` folder.

In [24]:
bio_data_df = get_biographic_data(SUBJECT_INDEX, names_of_subjects_folders)

subject_id_string = bio_data_df["subject_id_string"]
subject_id_number = bio_data_df["subject_id_number"]

Read thresholds from calibration data.

In [25]:
thresholds = get_thresholds(SUBJECT_INDEX, names_of_subjects_folders)
print(thresholds)

[0.53, 0.57, 0.58, 0.5]


Read eye-tracking data.

In [26]:
fix_df = get_eye_tracking_data(SUBJECT_INDEX, names_of_subjects_folders, thresholds)

In [27]:
fix_df.head()

,prop_dx_fixation,prop_sx_fixation,prop_center_fixation,block,trial,stim_left,stim_left_img_number,stim_right,stim_right_img_number,subject_id_number2
0,0.589286,0.142857,0.267857,5,107,orange,9,blue,4,0GLB5VNRFDP
1,0.346154,0.000000,0.653846,5,102,blue,5,orange,9,0GLB5VNRFDP
2,0.212766,0.446809,0.340426,1,14,blue,3,orange,7,0GLB5VNRFDP
3,0.312500,0.359375,0.328125,2,44,blue,7,orange,5,0GLB5VNRFDP
4,0.454545,0.151515,0.393939,3,74,orange,2,blue,5,0GLB5VNRFDP


In [28]:
# calculate mean of prop_dx_fixation column
mean_dx = fix_df['prop_dx_fixation'].mean()

# calculate mean of prop_sx_fixation column
mean_sx = fix_df['prop_sx_fixation'].mean()

# calculate mean of prop_center_fixation column
mean_center = fix_df['prop_center_fixation'].mean()

[mean_dx, mean_sx, mean_center]

[0.2894117593919631, 0.22672230664336093, 0.483865933964676]

Read PRL data

In [29]:
prl_df = read_prl_data(SUBJECT_INDEX, names_of_subjects_folders)


Merge the fixation DataFrame and the PRL DataFrame

In [30]:
tot_df = pd.merge(fix_df, prl_df, on="trial", how="outer")
tot_df.columns


Index(['prop_dx_fixation', 'prop_sx_fixation', 'prop_center_fixation', 'block',
       'trial', 'stim_left', 'stim_left_img_number', 'stim_right',
       'stim_right_img_number', 'subject_id_number2', 'more_rewarded_color',
       'stimulus_choice', 'blue_position', 'orange_position', 'left_img_name',
       'right_img_name', 'location_choice', 'keyboard_input', 'orange_reward',
       'blue_reward', 'outcome', 'start_reaction_time', 'end_reaction_time',
       'start_absolute_time', 'end_absolute_time'],
      dtype='object')

In [33]:
# Calculate rt column
tot_df["rt"] = tot_df["end_reaction_time"] - tot_df["start_reaction_time"]

# Create is_negative_img_chosen column
tot_df["is_negative_img_chosen"] = tot_df["stimulus_choice"].apply(
    lambda x: 1 if x == "blue" else 0
)

# Create feedback column
tot_df["feedback"] = tot_df["outcome"].apply(lambda x: 1 if x == "euro" else 0)

# Calculate mean feedback
mean_feedback = tot_df["feedback"].mean()
mean_feedback


0.72

In [34]:
tot_df.left_img_name

0      orange_9
1        blue_5
2        blue_3
3        blue_7
4      orange_2
         ...   
145      blue_3
146      blue_6
147    orange_6
148    orange_8
149    orange_1
Name: left_img_name, Length: 150, dtype: object

Recode `left_img_name` (orange_5, blue_8, ...) so as to keep only the name (discard the number).

In [35]:
left_img = tot_df["left_img_name"].str.split("_")
tot_df["image_on_left"] = pd.DataFrame(left_img.tolist())[0]

right_img = tot_df["right_img_name"].str.split("_")
tot_df["image_on_right"] = pd.DataFrame(right_img.tolist())[0]

Compute the proportion of left fixations, when the target is on the left and the proportion of right fixations, when the target is on the right. The sum of the two gives the total proportion of fixation to the target.

In [36]:
tot_df.shape

(150, 30)

In [37]:
tot_df.columns

Index(['prop_dx_fixation', 'prop_sx_fixation', 'prop_center_fixation', 'block',
       'trial', 'stim_left', 'stim_left_img_number', 'stim_right',
       'stim_right_img_number', 'subject_id_number2', 'more_rewarded_color',
       'stimulus_choice', 'blue_position', 'orange_position', 'left_img_name',
       'right_img_name', 'location_choice', 'keyboard_input', 'orange_reward',
       'blue_reward', 'outcome', 'start_reaction_time', 'end_reaction_time',
       'start_absolute_time', 'end_absolute_time', 'rt',
       'is_negative_img_chosen', 'feedback', 'image_on_left',
       'image_on_right'],
      dtype='object')

In [39]:
tot_df.head(2)

,prop_dx_fixation,prop_sx_fixation,prop_center_fixation,block,trial,stim_left,stim_left_img_number,stim_right,stim_right_img_number,subject_id_number2,...,outcome,start_reaction_time,end_reaction_time,start_absolute_time,end_absolute_time,rt,is_negative_img_chosen,feedback,image_on_left,image_on_right
0,0.589286,0.142857,0.267857,5,107,orange,9,blue,4,0GLB5VNRFDP,...,euro,503212,505557,11:50:56.889728,11:50:59.234327,2345,0,1,orange,blue
1,0.346154,0.000000,0.653846,5,102,blue,5,orange,9,0GLB5VNRFDP,...,euro,487492,488529,11:50:41.168602,11:50:42.205638,1037,0,1,blue,orange


In [43]:
avg_fix = tot_df.groupby(["stim_left", "stim_right"]).agg(
    avg_left_fix=("prop_sx_fixation", "mean"),
    avg_right_fix=("prop_dx_fixation", "mean"),
)
avg_fix

,,avg_left_fix,avg_right_fix
stim_left,stim_right,,
blue,orange,0.334230,0.165431
orange,blue,0.107102,0.427362


When the stimulus target (blue) is on the left, on the 0th row I have to compute:
`r0 = avg_left_fix / (avg_left_fix + avg_rigth_fix)`. When the stimulus target (blue) is on the right, on the 1st row I have to compute:
`r1 = avg_right_fix / (avg_left_fix + avg_rigth_fix)`. Then 
`blue_preference = (r0 + r1) / 2`.

In [56]:
blue_preference = avg_fix["avg_left_fix"][0] / (
    avg_fix["avg_left_fix"][0] + avg_fix["avg_right_fix"][0]
) + avg_fix["avg_right_fix"][1] / (
    avg_fix["avg_left_fix"][1] + avg_fix["avg_right_fix"][1]
)
blue_preference /= 2
blue_preference


0.734261144893066

A value of `blue_preference` greater than 0.5 indicates a preference for the blu stimulus, according to the proportion of fixations.

The blue stimulus had been chosen this proportion of times:

In [57]:
tot_df["is_negative_img_chosen"].mean()

0.52

It is still necessary to check the part below!!

In [59]:
tot_df["is_preference_right"] = tot_df["prop_dx_fixation"] - tot_df["prop_sx_fixation"]

fm = smf.ols("is_preference_right ~ stim_right", data=tot_df).fit()
print(fm.summary())

                             OLS Regression Results                            
Dep. Variable:     is_preference_right   R-squared:                       0.407
Model:                             OLS   Adj. R-squared:                  0.403
Method:                  Least Squares   F-statistic:                     101.5
Date:                 Sat, 18 Mar 2023   Prob (F-statistic):           1.67e-18
Time:                         11:51:37   Log-Likelihood:                -29.646
No. Observations:                  150   AIC:                             63.29
Df Residuals:                      148   BIC:                             69.31
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             

In [60]:
fm1 = smf.ols("is_preference_right ~ stim_left", data=tot_df).fit()
print(fm1.summary())

                             OLS Regression Results                            
Dep. Variable:     is_preference_right   R-squared:                       0.407
Model:                             OLS   Adj. R-squared:                  0.403
Method:                  Least Squares   F-statistic:                     101.5
Date:                 Sat, 18 Mar 2023   Prob (F-statistic):           1.67e-18
Time:                         11:51:40   Log-Likelihood:                -29.646
No. Observations:                  150   AIC:                             63.29
Df Residuals:                      148   BIC:                             69.31
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -

In [182]:
tot_df.groupby(["location_choice"]).agg(
    avg_left=("prop_sx_fixation", "mean"), avg_right=("prop_dx_fixation", "mean")
)

,avg_left,avg_right
location_choice,,
left,0.09229,0.602937
right,0.05719,0.686575


Average positive feedback.

In [183]:
mean_feedback = tot_df["feedback"].mean()
print(mean_feedback)

0.72
